# Training a CNN Model for ASL Detection

This notebook demonstrates how to build and train a Convolutional Neural Network (CNN) to classify static ASL signs. The trained model will be used for real-time detection of hand gestures.

## Import Required Libraries

This code cell imports all the necessary libraries for building and training the CNN model, including TensorFlow/Keras components.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

## Define Data Generators Function

This function creates training and validation data generators that will automatically load, preprocess, and batch the images for training.

In [2]:
def prepare_data_generators(data_dir, target_size=(224, 224), batch_size=32):
    """
    Prepare training and validation data generators.
    
    Args:
        data_dir (str): Path to the dataset directory
        target_size (tuple): Target image size for resizing
        batch_size (int): Batch size for training
        
    Returns:
        tuple: (train_generator, val_generator)
    """
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

## Define CNN Architecture

This function creates the Convolutional Neural Network architecture with multiple convolutional layers, pooling layers, and dropout for regularization.

In [3]:
def create_cnn_model(input_shape=(224, 224, 3), num_classes=26):
    """
    Create a CNN model for ASL classification.
    
    Args:
        input_shape (tuple): Shape of input images
        num_classes (int): Number of classes (26 for A-Z)
        
    Returns:
        tensorflow.keras.Model: Compiled CNN model
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## Load Preprocessed Data

This function loads the preprocessed dataset and prepares the data generators for training and validation.

In [4]:
def load_preprocessed_data(data_dir):
    """
    Load preprocessed dataset for training.
    
    Args:
        data_dir (str): Path to the dataset directory
        
    Returns:
        tuple: (train_generator, val_generator) - Data generators for training and validation
    """
    train_generator, val_generator = prepare_data_generators(data_dir)
    print("Data generators ready for training.")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    print(f"Number of classes: {train_generator.num_classes}")
    return train_generator, val_generator

## Train the Model

This function trains the CNN model using the prepared data generators and returns the training history for analysis.

In [ ]:
def train_model(model, train_generator, val_generator, epochs=10):
    """
    Train the CNN model.
    
    Args:
        model: The CNN model to train
        train_generator: Training data generator
        val_generator: Validation data generator
        epochs (int): Number of training epochs
        
    Returns:
        History: Training history object containing loss and accuracy metrics
    """
    print(f"Starting training for {epochs} epochs...")
    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(
        monitor='val_loss',      # You can also use 'val_accuracy'
        patience=2,             # Stop after 2 epochs with no improvement
        restore_best_weights=True
    )

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[early_stop],
        verbose=1
    )
    
    print("Training completed!")
    return history

## Save the Trained Model

This function saves the trained model to the specified path for later use in inference and real-time detection.

In [ ]:
def save_model(model, model_path):
    """
    Save the trained model to a specified path.
    
    Args:
        model: The trained CNN model
        model_path (str): Path where the model should be saved
    """
    model.save(model_path)
    print(f"Model saved successfully to: {model_path}")
    
    model_summary_path = model_path.replace('.h5', '_summary.txt')
    with open(model_summary_path, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"Model summary saved to: {model_summary_path}")

## Complete Training Pipeline Example

This cell demonstrates how to use all the functions together to train a complete ASL detection model.

In [7]:

# Configuration
DATA_DIR = "../data/asl_alphabet_train/asl_alphabet_train"  # Path to your dataset
MODEL_SAVE_PATH = "../models/asl_cnn_model.h5"
EPOCHS = 20

# Step 1: Load the data
data_generators = prepare_data_generators(DATA_DIR)
train_generator, val_generator = data_generators

# Step 2: Create the model
model = create_cnn_model(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)

# Step 3: Display model architecture
model.summary()

# Step 4: Train the model
history = train_model(model, train_generator, val_generator, epochs=EPOCHS)

# Step 5: Save the trained model
# Ensure the models directory exists before saving
os.makedirs("../models", exist_ok=True)
MODEL_SAVE_PATH = "../models/asl_cnn_model.h5"
model.save(MODEL_SAVE_PATH)
print(f"Model saved successfully to: {MODEL_SAVE_PATH}")

print("Training pipeline completed successfully!")

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,172,701 (42.62 MB)

 Trainable params: 11,172,701 (42.62 MB)

 Non-trainable params: 0 (0.00 B)

c:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Starting training for 20 epochs...
Epoch 1/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2005s 921ms/step - accuracy: 0.3228 - loss: 2.3638 - val_accuracy: 0.5697 - val_loss: 1.2738
Epoch 2/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2262s 1s/step - accuracy: 0.8115 - loss: 0.5542 - val_accuracy: 0.6660 - val_loss: 1.0503
Epoch 3/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2430s 1s/step - accuracy: 0.8880 - loss: 0.3214 - val_accuracy: 0.7218 - val_loss: 0.9190
Epoch 4/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2417s 1s/step - accuracy: 0.9213 - loss: 0.2266 - val_accuracy: 0.7268 - val_loss: 0.9630
Epoch 5/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2386s 1s/step - accuracy: 0.9387 - loss: 0.1765 - val_accuracy: 0.7224 - val_loss: 0.9107
Epoch 6/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2379s 1s/step - accuracy: 0.9492 - loss: 0.1470 - val_accuracy: 0.7327 - val_loss: 0.8836
Epoch 7/20
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2371s 1s/step - accuracy: 0.9574 - loss: 0.1268 - val_accuracy: 0.7562 - val_loss: 0.8817
Epoch 8/20
2175/2175 ━━━━━━━━━━━━━━━━━

Training completed!
Model saved successfully to: ../models/asl_cnn_model.h5
Training pipeline completed successfully!
